# 2.2 - Ajuste Supervisado

### GridSearching  -  CrossValidation


![grid](images/grid.png)

![cv](images/cv.ppm)

In [ ]:
import warnings
warnings.simplefilter('ignore')

import numpy as np


from sklearn.datasets import make_circles, make_regression

from sklearn.model_selection import train_test_split as tts


from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC


from sklearn.model_selection import GridSearchCV        # por fuerza bruta

In [ ]:
X = make_regression()[0]

y = make_regression()[1]

In [ ]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)

In [ ]:
modelo = RFR()

modelo.fit(X_train, y_train)


#R2
train_score = modelo.score(X_train, y_train)
test_score = modelo.score(X_test, y_test)

print ('Train: ',train_score)
print('Test: ', test_score)

In [ ]:
#help(RFR)

In [ ]:
params = {'n_estimators': [10, 20, 50, 100, 150, 200, 500],
          'max_depth': [5, 10, 20, 25]}

In [ ]:
def grid(modelo, params):

    g = GridSearchCV(modelo,                    # modelo de ML de sklearn
                     params,                    # dictio de parametros
                     cv=5,                      # cross-validation, 5 cortes
                     return_train_score=True,   # error de entrenamiento
                     n_jobs=-1                  # usa todos los cores
                    )


    g.fit(X, y)

    # ajuste de todos los modelos y combinaciones
    print('Acierto test: {:.2f}'.format(g.score(X_test, y_test)))
    print('Acierto train: {:.2f}'.format(g.score(X_train, y_train)))
    print('Mejores parametros: {}'.format(g.best_params_))
    print('Mejor acierto cv: {:.2f}'.format(g.best_score_))


    return g.best_estimator_.fit(X_train, y_train)

In [ ]:
%%time

modelo_grid = grid(RFR(), params)

In [ ]:
modelo_grid.score(X_train, y_train)

In [ ]:
modelo_grid.score(X_test, y_test)

### Random GridSearching

In [ ]:
from sklearn.model_selection import RandomizedSearchCV  # este es random

In [ ]:
X, y = make_circles(noise=0.1, factor=0.5, random_state=1)

In [ ]:
y[:10]

In [ ]:
import pylab as plt

plt.scatter(X[::, 0], X[::, 1], c=y);

In [ ]:
rfc = RFC()

In [ ]:
n_estimators=[int(x) for x in np.linspace(200, 2000, 10)]  # nº de arboles del bosque

min_samples_split=[2, 5, 10]   # minimo de elementos para splitear

bootstrap=[True, False]  # con o sin reemplazamiento

In [ ]:
params={'n_estimators': n_estimators,
        'min_samples_split':min_samples_split,
        'bootstrap': bootstrap}

In [ ]:
rf_random = RandomizedSearchCV(estimator=rfc,                  # modelo
                               param_distributions=params,    # dictio de parametros
                               
                               n_iter=200,  # nº max de iteraciones
                               
                               cv=3,        # nº de cortes
                               verbose=0,
                               n_jobs=-1,
                               scoring='roc_auc'
                              )

In [ ]:
rf_random.fit(X, y)

In [ ]:
rf_random.best_score_

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_estimator_.fit(X, y)

### HyperOpt (GridSearching bayesiano)

In [ ]:
import pandas as pd

import xgboost as xgb

from sklearn.metrics import mean_squared_error as mse

from sklearn.model_selection import train_test_split as tts

In [ ]:
df=pd.read_csv('../data/diamonds_train.csv')
df=df.dropna()
df.head()

In [ ]:
X=df.drop('price', axis=1)

y=df.price

In [ ]:
clarity={'I1':0, 'SI2':1, 'SI1':2, 'VS2':3, 'VS1':4,
         'VVS2':5, 'VVS1':6, 'IF':7}


cut={'Fair':0, 'Good':1, 'Very Good':2, 'Premium':3, 'Ideal':4}


color={'J':0, 'I':1, 'H':2, 'G':3, 'F':4, 'E':5, 'D':6}

In [ ]:
def label(s, dic):
    return dic[s]

In [ ]:
X.clarity=X.clarity.apply(lambda x: label(x, clarity))

X.cut=X.cut.apply(lambda x: label(x, cut))

X.color=X.color.apply(lambda x: label(x, color))

X.head()

In [ ]:
X_train, X_test, y_train, y_test = tts(X, y, 
                                       test_size=0.2, 
                                       random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
modelo = xgb.XGBRegressor()

modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

mse(y_test, y_pred, squared=False)

In [ ]:
modelo

In [ ]:
%pip install hyperopt

In [ ]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK

In [ ]:
# aprioris

space = {'n_estimators': hp.quniform('n_estimators', 10, 1000, 25),  # uniforme discreta
        
        'learning_rate': hp.uniform('learning_rate', 0.0001, 1.0),  # distribucion uniforme continua
    
        'max_depth': hp.quniform('max_depth', 4, 16, 1),

        'min_child_weight': hp.quniform('min_child', 1, 10, 1),

        'subsample': hp.uniform('subsample', 0.7, 1),

        'gamma': hp.uniform('gamma', 0.1, 0.5),

        'reg_lambda': hp.uniform('reg_lambda', 0, 1)
    }

In [ ]:
def objetivo(space):
    
    modelo = xgb.XGBRegressor(n_estimators=int(space['n_estimators']),
                             
                              learning_rate=space['learning_rate'],
                              
                              max_depth=int(space['max_depth']),

                              min_child_weight=space['min_child_weight'],

                              subsample=space['subsample'],

                              gamma=space['gamma'],

                              reg_lambda=space['reg_lambda'],

                              objective='reg:squarederror'
                             )
    
    eval_set = [(X_train, y_train), (X_test, y_test)]
    
    
    modelo.fit(X_train, y_train, eval_set=eval_set, eval_metric='rmse', verbose=False)
    

    y_pred = modelo.predict(X_test)
    
    rmse = mse(y_test, y_pred, squared=False)
    
    return {'loss': rmse, 'status': STATUS_OK}
    
    

In [ ]:
best = fmin(objetivo,
           space=space,
           algo=tpe.suggest,
           max_evals=10,
           trials=Trials())

best

In [ ]:
modelo=xgb.XGBRegressor(
    
    n_estimators=int(best['n_estimators']),
    
    learning_rate=best['learning_rate'],
    
    max_depth=int(best['max_depth']),
    
    subsample=best['subsample'],
    
    gamma=best['gamma'],
    
    reg_lambda=best['reg_lambda'],
    
    objective='reg:squarederror'
    )

In [ ]:
modelo.fit(X_train, y_train)

y_pred=modelo.predict(X_test)

mse(y_test, y_pred, squared=False)

In [ ]:
m=xgb.XGBRegressor()

m.fit(X_train, y_train)

y_pred=m.predict(X_test)

mse(y_test, y_pred, squared=False)